<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.62
    Uninstalling yfinance-0.2.62:
      Successfully uninstalled yfinance-0.2.62
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-06-09 14:14:11
-------------------
qualified stocks: 92
with latest results: 30
still star stocks: 17
-------------------
Initial Investment:  1.17 C
CY Investment:  1.44 C
Reserve:  1.01 L
Current:  1.35 C
-------------------
Today PnL: 1.67 L (1.25%)
Current PnL: -14.73 L (-10.26%)
CY Booked + Current PnL: -8.65 L (-6.03%)
-------------------
Total profit:  3.94 L
Total loss:  -18.67 L
-------------------
Total Booked + Current PnL: 18.33 L (15.73%)
Total Booked PnL: 33.06 L (28.37%)
Curr Year Booked PnL: 6.08 L (4.51%)
Prev Year Booked PnL: 26.98 L (23.15%)
Est FTT:  2.08 C
Est FTT PnL: 72.57 L (53.76%)
Deployed:  1.17 C
Current:  1.35 C
CAGR/XIRR %: 10.43%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,LICI,785.05,983.0,-2.96,H-LC,84.24,238340.0,43648.0,5434.0,0.12,22.42,2.28,25.21,34.0,8.03,1.78,31.80,XY25,NTT,INSURANCE
4,ADANIPORTS,1103.69,1583.0,4.33,M-LC,58.70,104526.0,26164.0,7871.0,0.03,33.39,7.53,43.43,77.0,3.32,0.78,39.80,XY24,NTT,MISC
61,RELIANCE,1291.83,1526.0,6.12,X-LC,34.24,157647.0,16838.0,8686.0,0.19,11.96,5.51,18.13,19.0,1.94,1.18,24.48,XY25,NTT,REFINERIES
84,UNIONBANK,123.87,163.0,1.23,M-LC,91.30,176383.0,35543.0,8943.0,1.21,25.24,5.07,31.59,88.0,3.97,1.32,53.21,XY24,NTT,BANKS
77,TATAELXSI,7332.28,7332.0,-11.61,X-MC,69.57,92820.0,-9832.0,9830.0,2.43,-9.58,10.59,-0.00,55.0,-1.00,0.69,39.52,OX40N,NTT,IT


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,COFFEEDAY,59.14,80.0,-57.75,L-SC,57.61,63494.0,-50055.0,90104.0,9.98,-44.08,141.91,35.27,260.0,-0.56,0.47,55.40,XR,NTT,HOTELS
47,JPPOWER,18.73,26.2,-33.95,L-SC,96.74,134648.0,-7194.0,63769.0,9.48,-5.07,47.36,39.88,256.0,-0.11,1.00,36.45,XY24,NTT,POWER
11,BANDHANBNK,214.25,400.0,-27.77,H-SC,79.35,215783.0,-32747.0,248215.0,7.19,-13.18,115.03,86.70,129.0,-0.13,1.61,42.75,XY24,NTT,BANKS
8,ASIANTILES,75.41,137.0,7011.11,M-SC,98.91,77999.0,-15811.0,92429.0,6.25,-16.85,118.50,81.67,235.0,-0.17,0.58,51.05,XR,NTT,CERAMICS
19,CERA,8421.60,8422.0,-11.93,X-SC,81.52,90883.0,-18598.0,18604.0,5.77,-16.99,20.47,0.00,64.0,-1.00,0.68,37.55,OX40N,NTT,CERAMICS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
14,BERGEPAINT,561.33,680.0,-15.79,H-MC,41.30,230830.0,3491.0,44573.0,-1.99,1.54,19.31,21.14,118.0,0.08,1.72,29.24,XY24,NTT,PAINTS
54,MEDANTA,1064.65,1486.0,3.34,X-MC,20.65,231253.0,25776.0,55547.0,-1.47,12.54,24.02,39.58,67.0,0.46,1.73,23.92,XY24,NTT,HEALTHCARE
62,REPCOHOME,515.07,880.0,-55.29,H-SC,54.35,165090.0,-39908.0,185148.0,-1.37,-19.47,112.15,70.85,135.0,-0.22,1.23,33.27,XY24,NTT,FINANCE
17,CAMPUS,294.86,393.0,NaN,NaN,86.96,168538.0,763.0,55078.0,-0.89,0.45,32.68,33.28,200.0,0.01,1.26,30.48,XY24,NTT,FOOTWEAR
73,STARHEALTH,551.56,761.0,3.44,M-SC,94.57,169158.0,-31058.0,107077.0,-0.85,-15.51,63.30,37.97,213.0,-0.29,1.26,35.82,XY24,NTT,INSURANCE


In [12]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,UJJIVANSFB,52.77,53.0,50.53,M-SC,77.17,130356.0,-12123.0,12749.0,0.94,-8.51,9.78,0.44,241.0,-0.95,0.97,53.86,OX40N,NTT,BANKS
77,TATAELXSI,7332.28,7332.0,-11.61,X-MC,69.57,92820.0,-9832.0,9830.0,2.43,-9.58,10.59,-0.00,55.0,-1.00,0.69,39.52,OX40N,NTT,IT
75,SYMPHONY,1306.42,1306.0,-27.85,M-SC,2.17,153728.0,-17413.0,17356.0,1.99,-10.17,11.29,-0.03,189.0,-1.00,1.15,10.49,OX40N,NTT,DURABLES
48,KANSAINER,299.63,340.0,-67.67,H-SC,7.61,225090.0,-44577.0,80920.0,0.38,-16.53,35.95,13.47,143.0,-0.55,1.68,12.66,XY24,NTT,PAINTS
19,CERA,8421.60,8422.0,-11.93,X-SC,81.52,90883.0,-18598.0,18604.0,5.77,-16.99,20.47,0.00,64.0,-1.00,0.68,37.55,OX40N,NTT,CERAMICS


In [13]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,TCS,3794.03,4998.00,-18.09,X-LC,10.87,267072.0,-28862.0,122773.0,1.18,-9.75,45.97,31.73,1.0,-0.24,1.99,6.90,X40,BTT,IT
43,INFY,1461.46,2275.00,-12.06,X-LC,40.22,281782.0,20181.0,125449.0,0.67,7.71,44.52,55.67,2.0,0.16,2.10,14.29,X40,BTT,IT
52,LTIM,5564.16,7197.33,-1.63,X-LC,67.39,189360.0,-10950.0,69741.0,1.08,-5.47,36.83,29.35,3.0,-0.16,1.41,30.82,X200,ATH,IT
7,ASIANPAINT,2961.56,4250.00,-23.13,X-LC,8.70,152857.0,-48529.0,136150.0,0.12,-24.10,89.07,43.51,5.0,-0.36,1.14,5.43,X40,BTT,PAINTS
35,HINDUNILVR,2413.81,2723.00,-14.69,X-LC,21.74,267837.0,-2510.0,37149.0,0.07,-0.93,13.87,12.81,6.0,-0.07,2.00,10.85,XY25,NTT,FMCG


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,UNIONBANK,123.87,163.0,1.23,M-LC,91.30,176383.0,35543.0,8943.0,1.21,25.24,5.07,31.59,88.0,3.97,1.32,53.21,XY24,NTT,BANKS
4,ADANIPORTS,1103.69,1583.0,4.33,M-LC,58.70,104526.0,26164.0,7871.0,0.03,33.39,7.53,43.43,77.0,3.32,0.78,39.80,XY24,NTT,MISC
51,LICI,785.05,983.0,-2.96,H-LC,84.24,238340.0,43648.0,5434.0,0.12,22.42,2.28,25.21,34.0,8.03,1.78,31.80,XY25,NTT,INSURANCE


In [15]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SAIL,130.64,228.00,102.64,M-MC,85.87,230886.0,5924.0,161736.0,-0.71,2.63,70.05,74.53,183.0,0.04,1.72,34.05,XY24,BTT,STEEL
36,HINDZINC,514.80,744.74,40.53,M-LC,93.48,104870.0,1910.0,44077.0,4.40,1.86,42.03,44.67,68.0,0.04,0.78,35.33,X5K,ATH,METALS
18,CAMS,3643.00,5250.99,6.89,H-SC,88.04,120159.0,18155.0,26868.0,0.69,17.80,22.36,44.14,125.0,0.68,0.90,36.93,X40N,ATH,MISC
4,ADANIPORTS,1103.69,1583.00,4.33,M-LC,58.70,104526.0,26164.0,7871.0,0.03,33.39,7.53,43.43,77.0,3.32,0.78,39.80,XY24,NTT,MISC
84,UNIONBANK,123.87,163.00,1.23,M-LC,91.30,176383.0,35543.0,8943.0,1.21,25.24,5.07,31.59,88.0,3.97,1.32,53.21,XY24,NTT,BANKS


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
90,WHIRLPOOL,1167.49,2270.00,-31.38,M-SC,53.26,200214.0,28593.0,133483.0,5.71,16.66,66.67,94.43,191.0,0.21,1.49,47.77,XR,NTT,DURABLES
12,BANKINDIA,116.91,190.00,-25.79,M-MC,73.91,198187.0,18379.0,94040.0,3.60,10.22,47.45,62.52,168.0,0.20,1.48,42.62,XR,NTT,BANKS
40,INDIAMART,2327.09,4871.06,-50.53,H-SC,47.83,128790.0,5454.0,129382.0,0.08,4.42,100.46,109.32,122.0,0.04,0.96,27.24,AR,ATH,MISC
36,HINDZINC,514.80,744.74,40.53,M-LC,93.48,104870.0,1910.0,44077.0,4.40,1.86,42.03,44.67,68.0,0.04,0.78,35.33,X5K,ATH,METALS
91,WIPRO,243.46,420.00,-9.26,M-LC,23.91,155186.0,4241.0,105216.0,0.69,2.81,67.80,72.51,70.0,0.04,1.16,7.42,XR,NTT,IT


In [17]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,COFFEEDAY,59.14,80.00,-57.75,L-SC,57.61,63494.0,-50055.0,90104.0,9.98,-44.08,141.91,35.27,260.0,-0.56,0.47,55.40,XR,NTT,HOTELS
5,ALKYLAMINE,2347.98,4567.52,-7.04,X-SC,89.13,86516.0,-14447.0,109884.0,1.53,-14.31,127.01,94.53,59.0,-0.13,0.65,28.27,SR,ATH,CHEMICALS
53,MASFIN,326.60,399.50,-20.05,H-SC,72.83,90645.0,-7335.0,29206.0,2.51,-7.49,32.22,22.32,138.0,-0.25,0.68,31.47,XR,ATH,FINANCE
4,ADANIPORTS,1103.69,1583.00,4.33,M-LC,58.70,104526.0,26164.0,7871.0,0.03,33.39,7.53,43.43,77.0,3.32,0.78,39.80,XY24,NTT,MISC
36,HINDZINC,514.80,744.74,40.53,M-LC,93.48,104870.0,1910.0,44077.0,4.40,1.86,42.03,44.67,68.0,0.04,0.78,35.33,X5K,ATH,METALS


In [18]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
86,VALIANTORG,512.64,838.00,-369.49,H-SC,100.00,114699.0,-18587.0,103183.0,2.19,-13.95,89.96,63.47,144.0,-0.18,0.86,92.26,XR,NTT,CHEMICALS
8,ASIANTILES,75.41,137.00,7011.11,M-SC,98.91,77999.0,-15811.0,92429.0,6.25,-16.85,118.50,81.67,235.0,-0.17,0.58,51.05,XR,NTT,CERAMICS
80,TITAGARH,1097.23,1548.00,-4.11,H-SC,97.83,168595.0,-23420.0,102303.0,2.39,-12.20,60.68,41.08,152.0,-0.23,1.26,42.69,XY24,NTT,ENGINEERING
47,JPPOWER,18.73,26.20,-33.95,L-SC,96.74,134648.0,-7194.0,63769.0,9.48,-5.07,47.36,39.88,256.0,-0.11,1.00,36.45,XY24,NTT,POWER
76,TANLA,917.30,1963.11,-33.39,H-SC,95.65,163712.0,-75703.0,348657.0,0.71,-31.62,212.97,114.01,127.0,-0.22,1.22,50.15,AR,ATH,IT


In [19]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.39
1,25,43.23
2,50,71.90


In [20]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     33.42
XY25     15.20
XR       11.30
X40      10.72
X40N      8.36
OX40N     8.26
AR        7.86
X5K       2.30
X200      1.41
SR        1.17
Name: CurrAlloc%, dtype: float64

In [21]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    18.38
X-LC    17.84
M-SC    16.45
M-LC     9.97
H-MC     8.85
H-LC     7.77
X-MC     6.14
M-MC     5.84
L-SC     3.12
X-SC     2.76
L-LC     1.11
Name: CurrAlloc%, dtype: float64

In [22]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.00,-0.88,34.45
IT,10.88,-13.92,71.76
BANKS,10.30,-5.04,52.12
MISC,6.39,-26.41,75.84
FINANCE,6.30,-19.19,65.05
PAINTS,5.55,-17.30,40.43
INSURANCE,5.22,4.61,23.89
ELECTRICAL,5.11,-2.75,47.32
HEALTHCARE,4.63,-3.75,34.06


In [23]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2847820.0,26
XR,1198284.0,14
AR,1016606.0,9
X40,612984.0,8
XY25,494904.0,10
X40N,409174.0,8
OX40N,324904.0,11
SR,188974.0,2
X5K,93758.0,3


In [24]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,1903726.0,18
M-SC,1671148.0,20
X-LC,821439.0,11
M-MC,561870.0,5
H-MC,542567.0,7
M-LC,380032.0,8
X-MC,323716.0,5
L-SC,308950.0,5
X-SC,263854.0,4


In [25]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,,sum,count
Conviction,Criteria,,
H-SC,XY24,889401.0,6
M-SC,XY24,730715.0,6
H-SC,AR,537277.0,3
X-LC,X40,466561.0,4
M-SC,XR,415035.0,5
M-MC,XY24,382965.0,3
H-MC,XY24,223959.0,2
H-SC,XR,206103.0,3
M-LC,XY24,202039.0,4


In [26]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 31.0 seconds
